### 1. Load Packages

In [1]:
import openai
import pandas as pd
import re
import json

### 2. Load Tokens

In [2]:
openai.api_key = "To be filled"

### 3. Load Data or Text

In [3]:
df1 = pd.read_csv('processed_60900_61899.csv')
df2 = pd.read_csv('processed_177000_177999.csv')

In [4]:
df1

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,60900,60900,6555,we already have congestion pricing for taxis &...,2021-07-09 00:02:34+00:00,2021-07-09,2021-27
1,60901,60901,6556,an the best cannot be that much i am sorry htt...,2021-07-08 23:44:03+00:00,2021-07-08,2021-27
2,60902,60902,6557,handicap uber's is getting out of hand,2021-07-08 23:40:10+00:00,2021-07-08,2021-27
3,60903,60903,6558,bitchh i would calmly get in a cab laugh ill b...,2021-07-08 23:16:21+00:00,2021-07-08,2021-27
4,60904,60904,6559,take a cab. you are getting hepatitis walking ...,2021-07-08 23:15:04+00:00,2021-07-08,2021-27
...,...,...,...,...,...,...,...
995,61895,61895,7550,so...i confess ...i walked to target &amp; bou...,2021-06-17 23:27:09+00:00,2021-06-17,2021-24
996,61896,61896,7551,"uber, ferry, plane, train today",2021-06-17 23:00:01+00:00,2021-06-17,2021-24
997,61897,61897,7552,"hi, i order some food about a hour ago and i h...",2021-06-17 22:08:29+00:00,2021-06-17,2021-24
998,61898,61898,7553,i was there. it was an uber. none of the other...,2021-06-17 21:54:38+00:00,2021-06-17,2021-24


In [5]:
df2

,Unnamed: 0,GLOBAL_ID,LOCAL_ID,processed_txt,date,_date_,WoY
0,177000,177000,7102,please kick japan out of my country they are s...,2021-11-26 13:17:13+00:00,2021-11-26,2021-47
1,177001,177001,7103,once i learned about correctional exercises an...,2021-11-26 12:41:27+00:00,2021-11-26,2021-47
2,177002,177002,7104,empty train cars &gt;&gt;,2021-11-26 12:22:52+00:00,2021-11-26,2021-47
3,177003,177003,7105,that emerald parking lot is about 1 mile walk ...,2021-11-26 12:14:05+00:00,2021-11-26,2021-47
4,177004,177004,7106,"that is going to take time, i need car by end ...",2021-11-26 11:48:54+00:00,2021-11-26,2021-47
...,...,...,...,...,...,...,...
995,177995,177995,8097,how about getting all trash off the sidewalk? ...,2021-06-03 02:03:54+00:00,2021-06-03,2021-22
996,177996,177996,8098,get trash off the sidewalk. trash corals in th...,2021-06-03 02:00:17+00:00,2021-06-03,2021-22
997,177997,177997,8099,people of philadelphia find trae young's hotel...,2021-06-03 01:55:00+00:00,2021-06-03,2021-22
998,177998,177998,8100,ring rain coming back for more and do not you ...,2021-06-03 01:40:50+00:00,2021-06-03,2021-22


### 4.baseline

In [6]:
def generate_base(tweet):
    tweet = tweet.lower()  # Ensure the tweet is in lowercase
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {
                "role": "system",
                "content": "You are an AI trained to extract travel experiences from tweets."
            },
            {
                "role": "user",
                "content": f"Given the tweet: \"{tweet}\" 1. What is the corresponding travel mode? (Train, subway, bus, bike, taxi, Uber, private car) 2. Is the user satisfied with the corresponding travel mode’s service? (Yes or No) 3. If not satisfied, what are the reasons behind?"
                   "Conclude with a concise, structured summary (keep this format for all your response) in JSON format,like this: "
                   "{\"travel_mode\": \"<inferred mode>\", \"sentiment\": \"<inferred sentiment>\", \"reasons\": \"<inferred reasons>\"}."
            }
        ],
        temperature=0.5,
        max_tokens=250
    )
    
    return response.choices[0].message.content if response.choices else "No response"

In [7]:
def parse_response(response_text):
    # Parse the JSON text and extract necessary information
    try:
        data = json.loads(response_text)
        return data.get('travel_mode', ''), data.get('sentiment', ''), data.get('reasons', '')
    except json.JSONDecodeError:
        return '', '', ''  # Return empty strings if JSON is invalid

In [8]:
# test
tweet = df1.iloc[0]['processed_txt']
response_text = generate_base(tweet)
travel_mode, sentiment, reasons = parse_response(response_text)
travel_mode, sentiment, reasons

('Mass transit (train, subway, bus)',
 'Satisfied',
 'The user supports congestion pricing to encourage the use of mass transit, citing examples of reduced traffic in London and Stockholm.')

In [9]:
data_list = []
# Iterate through the original DataFrame
for index, row in df1.iterrows():
    GLOBAL_ID = row['GLOBAL_ID']
    tweet = row['processed_txt']  # Assuming 'processed_txt' contains the tweet text
    response_text = generate_base(tweet)
    travel_mode, sentiment, reasons = parse_response(response_text)
    
    # Append the results to the list
    data_list.append({'GLOBAL_ID': GLOBAL_ID, 'Tweet': tweet, 'Travel Mode': travel_mode, 'Sentiment': sentiment, 'Reasons': reasons})

# Create the DataFrame from the accumulated list
df1_res = pd.DataFrame(data_list)

# Save the new DataFrame to a CSV file
df1_res.to_csv('baseline_df1.csv', index=False)
df1_res

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,60900,we already have congestion pricing for taxis &...,private car,No,The user is not satisfied with the private car...
1,60901,an the best cannot be that much i am sorry htt...,unknown,unknown,The tweet does not provide clear information t...
2,60902,handicap uber's is getting out of hand,Uber,No,The user is not satisfied with Uber's service ...
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,No,The user seems dissatisfied with the taxi serv...
4,60904,take a cab. you are getting hepatitis walking ...,taxi,No,Unsafe environment and dissatisfaction with th...
...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,Uber,Yes,
996,61896,"uber, ferry, plane, train today","Multi-modal (Uber, ferry, plane, train)",Neutral,N/A
997,61897,"hi, i order some food about a hour ago and i h...",food delivery service,No,Delay in receiving the food order
998,61898,i was there. it was an uber. none of the other...,Uber,Neutral,Not applicable


In [13]:
data_list = []
# Iterate through the original DataFrame
for index, row in df2.iterrows():
    GLOBAL_ID = row['GLOBAL_ID']
    tweet = row['processed_txt']  # Assuming 'processed_txt' contains the tweet text
    response_text = generate_base(tweet)
    travel_mode, sentiment, reasons = parse_response(response_text)
    
    # Append the results to the list
    data_list.append({'GLOBAL_ID': GLOBAL_ID, 'Tweet': tweet, 'Travel Mode': travel_mode, 'Sentiment': sentiment, 'Reasons': reasons})

# Create the DataFrame from the accumulated list
df2_res = pd.DataFrame(data_list)

# Save the new DataFrame to a CSV file
df2_res.to_csv('baseline_df2.csv', index=False)
df2_res

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons
0,177000,please kick japan out of my country they are s...,private car,No,The user is not satisfied with the private car...
1,177001,once i learned about correctional exercises an...,car,N/A,N/A
2,177002,empty train cars &gt;&gt;,train,positive,"User is satisfied with the empty train cars, i..."
3,177003,that emerald parking lot is about 1 mile walk ...,shuttle bus,No,Nightmare leaving the lot if you do not leave ...
4,177004,"that is going to take time, i need car by end ...",private car,Neutral,N/A
...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,private car,No,User is not satisfied with the service due to ...
996,177996,get trash off the sidewalk. trash corals in th...,private car,No,User is not satisfied with the service due to ...
997,177997,people of philadelphia find trae young's hotel...,private car,No,The user is not satisfied with the private car...
998,177998,ring rain coming back for more and do not you ...,private car,Neutral,N/A


### 5. Self-Verification

In [14]:
def self_verify(tweet, travel_mode, sentiment, reasons):
    verification_prompt = f"""
    Based on the following tweet analysis:
    Tweet: {tweet}
    - Travel Mode: {travel_mode}
    - Sentiment: {sentiment}
    - Reasons: {reasons}

    Is this analysis 'correct' or 'incorrect'? Consider the relevance and accuracy of the travel mode, sentiment, and reasons given.
    """
    
    response = openai.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{
            "role": "system",
            "content": "You're a highly intelligent assistant. Please read the following analysis carefully and provide your verdict."
        }, {
            "role": "user",
            "content": verification_prompt
        }],
        temperature=0,  # Use 0 for deterministic output
        max_tokens=50
    )

    # Simplifying the response check
    text_response = response.choices[0].message.content.strip().lower()
    if "correct" in text_response:
        return "correct"
    elif "incorrect" in text_response:
        return "incorrect"
    else:
        return "undetermined"  # Use this if the response isn't clear


In [15]:
#test
self_verify(df1_res.iloc[0]['Tweet'],df1_res.iloc[0]['Travel Mode'], df1_res.iloc[0]['Sentiment'], df1_res.iloc[0]['Reasons'])

'correct'

**df1**

In [16]:
#df1
df1_res['Self Verification Result'] = df1_res.apply(
    lambda row: self_verify(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [17]:
df1_res

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Self Verification Result
0,60900,we already have congestion pricing for taxis &...,private car,No,The user is not satisfied with the private car...,correct
1,60901,an the best cannot be that much i am sorry htt...,unknown,unknown,The tweet does not provide clear information t...,correct
2,60902,handicap uber's is getting out of hand,Uber,No,The user is not satisfied with Uber's service ...,correct
3,60903,bitchh i would calmly get in a cab laugh ill b...,taxi,No,The user seems dissatisfied with the taxi serv...,correct
4,60904,take a cab. you are getting hepatitis walking ...,taxi,No,Unsafe environment and dissatisfaction with th...,correct
...,...,...,...,...,...,...
995,61895,so...i confess ...i walked to target &amp; bou...,Uber,Yes,,correct
996,61896,"uber, ferry, plane, train today","Multi-modal (Uber, ferry, plane, train)",Neutral,N/A,correct
997,61897,"hi, i order some food about a hour ago and i h...",food delivery service,No,Delay in receiving the food order,correct
998,61898,i was there. it was an uber. none of the other...,Uber,Neutral,Not applicable,correct


In [18]:
df1_res.to_csv('Week 11 result_60900_61899(baseline).csv', index=False)

In [19]:
df1_res['Self Verification Result'].value_counts().get('correct', 0)

990

**df2**

In [20]:
#df2
df2_res['Self Verification Result'] = df2_res.apply(
    lambda row: self_verify(row['Tweet'], row['Travel Mode'], row['Sentiment'], row['Reasons']), axis=1)

In [21]:
df2_res

,GLOBAL_ID,Tweet,Travel Mode,Sentiment,Reasons,Self Verification Result
0,177000,please kick japan out of my country they are s...,private car,No,The user is not satisfied with the private car...,correct
1,177001,once i learned about correctional exercises an...,car,N/A,N/A,correct
2,177002,empty train cars &gt;&gt;,train,positive,"User is satisfied with the empty train cars, i...",correct
3,177003,that emerald parking lot is about 1 mile walk ...,shuttle bus,No,Nightmare leaving the lot if you do not leave ...,correct
4,177004,"that is going to take time, i need car by end ...",private car,Neutral,N/A,correct
...,...,...,...,...,...,...
995,177995,how about getting all trash off the sidewalk? ...,private car,No,User is not satisfied with the service due to ...,correct
996,177996,get trash off the sidewalk. trash corals in th...,private car,No,User is not satisfied with the service due to ...,correct
997,177997,people of philadelphia find trae young's hotel...,private car,No,The user is not satisfied with the private car...,correct
998,177998,ring rain coming back for more and do not you ...,private car,Neutral,N/A,correct


In [22]:
df2_res.to_csv('Week 11 result_177000_177999(baseline).csv', index=False)

In [23]:
df2_res['Self Verification Result'].value_counts().get('correct', 0)

993

## 6. Result Evaluation

In [5]:
def analyze_data(df):
    # Analyze 'Travel Mode'
    unique_travel_modes = df['Travel Mode'].unique()
    print("Distinct outcomes for Travel Mode:", unique_travel_modes)

    number_of_distinct_travel_modes = df['Travel Mode'].nunique()
    print("Number of distinct outcomes for Travel Mode:", number_of_distinct_travel_modes)

    travel_mode_counts = df['Travel Mode'].value_counts()
    print("Counts for each distinct outcome in Travel Mode:", travel_mode_counts)

    # Analyze 'Sentiment'
    unique_sentiments = df['Sentiment'].unique()
    print("Distinct outcomes for Sentiment:", unique_sentiments)

    number_of_distinct_sentiments = df['Sentiment'].nunique()
    print("Number of distinct outcomes for Sentiment:", number_of_distinct_sentiments)

    sentiment_counts = df['Sentiment'].value_counts()
    print("Counts for each distinct outcome in Sentiment:", sentiment_counts)

    # Calculate various rates
    extraction_rate = len(df[~df['Travel Mode'].isin(['na', 'NA'])]) / len(df)
    satisfaction_rate = len(df[df['Sentiment'] == 'positive']) / len(df)
    self_verification_rate = len(df[df['Self Verification Result'] == 'correct']) / len(df)
    
    if 'Cross Verification Result' in df.columns:
        cross_verification_rate = len(df[df['Cross Verification Result'] == 'correct']) / len(df)
    else:
        cross_verification_rate = None
    
    return {
        'Extraction Rate': extraction_rate,
        'Satisfaction Rate': satisfaction_rate,
        'Self Verification Rate': self_verification_rate,
        'Cross Verification Rate': cross_verification_rate
    }


### (1) DF1

In [8]:
df1_res = pd.read_csv('Week 11 result_60900_61899(baseline).csv')

In [9]:
analyze_data(df1_res)

Distinct outcomes for Travel Mode: ['private car' 'unknown' 'Uber' 'taxi' 'subway' 'Lyft car service' 'beach'
 'Bus' 'taxi/Uber' 'ridesharing (Lyft)' 'bike' 'food delivery service'
 'Taxi' 'train' 'flight' 'betting' 'plane' 'bus' 'restaurant' 'walking'
 'Tesla Model' 'Bike' 'renting a car' 'Lyft' 'Taxi or Uber'
 'Ride-sharing service (Uber)' 'taxi (Lyft)' 'Airplane' 'hansom cab'
 'Ride-sharing (Lyft)' 'company' 'delivery service'
 'food delivery service (Doordash)' 'unclear' 'ride' 'ride (unspecified)'
 'Unknown' 'van' 'Airport' 'guide' 'ride-sharing service (Uber or taxi)'
 'Subway' 'Uncertain' 'flights' 'None' 'ride-sharing (Lyft)'
 'Rideshare (Lyft)' 'car' 'Not applicable' 'skating' 'pool' 'packing' nan
 'food delivery service (e.g., Uber Eats)' 'public transit' 'Airline'
 'poolside' 'Train' 'public transportation (MBTA and commuter rail)'
 'train or Uber' 'airplane' 'bus and train' 'Train, taxi, rental car'
 'food delivery service (e.g. Doordash)' 'Megabus'
 'public transportation 

{'Extraction Rate': 1.0,
 'Satisfaction Rate': 0.018,
 'Self Verification Rate': 0.99,
 'Cross Verification Rate': None}

### (2) DF2

In [10]:
df2_res = pd.read_csv('Week 11 result_177000_177999(baseline).csv')

In [11]:
analyze_data(df2_res)

Distinct outcomes for Travel Mode: ['private car' 'car' 'train' 'shuttle bus' 'subway'
 'public transportation (subway/train/bus)'
 'public transportation (subway)' 'bike' 'convoy' 'commuter train'
 'car rental' 'public transportation' 'taxi'
 'ride-sharing service (Lyft/Uber)' 'scooter' 'electric car'
 'outdoor dining' 'Uber' 'Train' 'used car salesman' 'bus' 'car bomb'
 'unknown' 'Flights, hotels, car insurance, Verizon FiOS, etc.'
 'helicopter' 'motorcycle' 'Subway' 'Walking' 'car ride'
 'delivery service' 'Brewery visit' 'vehicle' 'cars' 'flying cars'
 'skates' 'drive' 'electric scooter' nan 'not owning a car' 'walking'
 'car service (like Uber or private car)' 'rental car' 'pedestrian'
 'transit car (train, subway, bus)' 'Taxi' 'Degree' 'scooter share'
 'stock trading' 'renting a car' 'boat' 'Walking, biking, rail, bus'
 'horses' 'city where you do not need a car' 'Private car'
 'No vehicle rental' 'public transportation (train or bus)'
 'public transportation (subway or bus)' 'pl

{'Extraction Rate': 1.0,
 'Satisfaction Rate': 0.022,
 'Self Verification Rate': 0.993,
 'Cross Verification Rate': None}